In [42]:
import os
import cv2
import random
import matplotlib.pyplot as plt
from natsort import natsorted
from uuid import uuid4

In [43]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        print(f"Failed to read {image_path}")
    return image

In [44]:
def save_image_and_labels(labels, image, image_shape, i):
    h, w = image_shape[:2]
    id = str(uuid4())

    os.makedirs("labels", exist_ok=True)
    os.makedirs("images", exist_ok=True)

    with open(f"labels/{id}-pokemon_{i}.txt", "w") as f:
        for card_id, x1, y1, x2, y2 in labels:
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            box_width = (x2 - x1) / w
            box_height = (y2 - y1) / h
            f.write(f"{card_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

    cv2.imwrite(f"images/{id}-pokemon_{i}.png", image)

In [45]:
def draw_random_cards(blank_table, cards_dict, card_keys: list, area, num_cards):
    table = blank_table.copy()
    labels = []

    x_min, y_min, x_max, y_max = area
    for _ in range(num_cards):
        if len(cards_dict) == 0:
            break

        card_key = random.choice(list(cards_dict.keys()))
        card_id = card_keys.index(card_key)

        card_path = random.choice(cards_dict[card_key])
        cards_dict[card_key].remove(card_path) 
        if len(cards_dict[card_key]) == 0:
            cards_dict.pop(card_key)

        card = load_image(card_path)
        if card is None:
            continue

        # Add alpha if missing
        if card.shape[2] == 3:
            card = cv2.cvtColor(card, cv2.COLOR_BGR2BGRA)

        # Random size
        target_w, target_h = 100, 140
        card = cv2.resize(card, (target_w, target_h), interpolation=cv2.INTER_AREA)

        # Random rotation
        angle = random.choice(range(-30, 31, 5))
        center = (target_w // 2, target_h // 2)
        rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)

        cos = abs(rot_mat[0, 0])
        sin = abs(rot_mat[0, 1])
        bound_w = int((target_h * sin) + (target_w * cos))
        bound_h = int((target_h * cos) + (target_w * sin))

        rot_mat[0, 2] += (bound_w / 2) - center[0]
        rot_mat[1, 2] += (bound_h / 2) - center[1]

        rotated_card = cv2.warpAffine(card, rot_mat, (bound_w, bound_h),
                                      flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT,
                                      borderValue=(0, 0, 0, 0))

        # Random position inside the allowed area
        max_x = x_max - bound_w
        max_y = y_max - bound_h
        if max_x <= x_min or max_y <= y_min:
            continue
        offset_x = random.randint(x_min, max_x)
        offset_y = random.randint(y_min, max_y)

        roi = table[offset_y:offset_y+bound_h, offset_x:offset_x+bound_w]

        alpha = rotated_card[:, :, 3] / 255.0
        for c in range(3):
            roi[:, :, c] = (1 - alpha) * roi[:, :, c] + alpha * rotated_card[:, :, c]
        table[offset_y:offset_y+bound_h, offset_x:offset_x+bound_w] = roi

        # Save bounding box
        labels.append((card_id, offset_x, offset_y, offset_x + bound_w, offset_y + bound_h))

    return table, labels

In [46]:
import os
from glob import glob

card_dir = "cards"
card_dict = {}

for card_name in os.listdir(card_dir):
    full_path = os.path.join(card_dir, card_name)
    if os.path.isdir(full_path):
        images = glob(os.path.join(full_path, "*"))
        card_dict[card_name] = images


In [47]:
card_list = natsorted(os.listdir(card_dir))

In [48]:
blank_table = load_image('table.png')
area = (100, 100, blank_table.shape[1] - 100, blank_table.shape[0] - 100)

i = 0
while(len(card_dict) > 0):
    table_with_cards, labels = draw_random_cards(blank_table=blank_table, cards_dict=card_dict, card_keys=card_list, area=area, num_cards=random.randint(3,15))
    
    save_image_and_labels(labels, table_with_cards, blank_table.shape, i)
    i += 1

In [49]:
with open("classes.txt", "w") as f:
    for card in card_list:
        f.write(f"{card}\n")